# Practical Machine Learning with SDSS Data

In this tutorial, we are going to use SDSS data to get some hands-on experience with machine learning. In the first exercise, we're going to train a classifier to distinguish stars from galaxies. This is an important problem, because many scientific questions depends on a clean data sets, and stars can be hard to distinguish from far-away galaxies. We don't want our study of galaxy evolution contaminated by stars in our own galaxy!

As a note in advance: none of the results you'll get out of this are science-worthy. This tutorial is meant to give you a first idea for how to set up your own machine learning model. But the first, and most important lesson, is this: **don't blindly trust your ML results.** 
As with any other science project, reporting or using results from a machine learning classifier or regressor requires careful understanding of the biases and caveats, assumptions and limitations that come with the data and algorithms chosen. Because the data sets you'll be using come straight out of the SDSS catalogue, you can expect there to be funny effects (both subtle and not) that may mess up any classification you'd want to do, and in a real-world setting, this would include understanding the limitations of the instrument and the data processing, before drawing any scientific conclusions from your procedure. 

With that out of the way, let's have some fun with machine learning! In this tutorial, we will use python and a library called `scikit-learn` to do our machine learning, `pandas` to deal with data structures, and `matplotlib` and `seaborn` to do our plotting. 

In [1]:
# make plots interactive and import plotting functionality
%matplotlib notebook
import matplotlib.pyplot as plt

# pretty plotting
import seaborn as sns

# my standard styles for plots
sns.set_style("whitegrid")
sns.set_context("talk")

# Always need numpy
import numpy as np

# data array operations
import pandas as pd

### Load the Data

Our first task is loading the data. For this exercise, we'll work with `sdss_dataset1.csv`. Your task is to find the correct file in this folder and load the data into a `pandas.DataFrame` (if you've never worked with pandas, take a look at the `read_csv` function):

In [ ]:
data =    # add your code here

The `head` method on your loaded `DataFrame` gives you a quick overview of what's in your data.

**Exercise**: What columns do you recognize? Which ones are new to you? Which columns do you think will be su

Some quick lingo: In machine learning, the things we are trying to learn are often called **labels**, and the quantities we can use to learn them are **features**. For example, in some of the data sets, you're going to try and separate stars and galaxies by their magnitudes and colours. Here, for each **sample** in your data set, you have a bunch of magnitude and colour measurements, your features, and you're trying to predict whether that sample is a galaxy or a star, its label. For the photometric redshift estimation case, you similarly have magnitudes and colours as features, and you're trying to predict the redshifts (your labels). This is called **supervised learning**. 

Note that in this case, we always need examples where we *know* the ground truth: we need to know the class really well, or we need to know the redshift beyond a reasonable doubt (in our case here e.g. through precise spectroscopic measurements). This is often not the case in astronomy (or, indeed, science): we often don't know exactly what our labels should be. In these cases, **unsupervised learning** can be really helpful. Some of you have data sets without labels. You'll be playing around with clustering algorithms.

## Machine Learning With Messy Data

Let's start with something I told you *not* to do in the earlier class: we're just going to build a classifier and see how it does, without knowing too much of what's in the data. 

Normally, you wouldn't *start* by doing a classification, but for most of your data sets, there are some points we're going to make throughout this tutorial, so having a classification without knowing much about the data serves as a useful baseline. In general, though, running an ML algorithm comes at the end of *many* important steps, which is part of the point of this entire tutorial.

### Splitting the Data into a Training and Test Set

Our first task is to split the data into a training and a test data set. Pick the first 51% of the data and designate this as our training data set, and the remaining 49% of the DataFrame will be our test data set. This assigns a way higher fraction of data to the test data set than one normally would, but bear with me here. This entire example is a little contrived in the service of being instructional. :)

In [ ]:
ntraining = # number of training examples
ntest = # number of test examples

Note: the thing you're classifying on should *not* be part of the array you use to classify, so remove the `class` column from the training arrays (but make sure you store it in a separate array first! (Hint: the pandas `drop` method comes in handy here)

**Advice**: It may be tempting to store the column with the label `class` in a variable `class`. Please don't! `class` is one of python's protected variable names, because it's used to set up a class. Similarly, you should not name a variable `def`: python might let you do it, but then you can never ever make a function again, because you used the keyword Python uses to create a function (i.e. when you type `def myfunction(...): ...` into a variable! There are a few of those to look out for, other examples include `list` and `lambda`. You will not believe how often I've named a list, well `list`, and could then no longer make any lists! Don't make my mistakes! :) 

In [21]:
classes = # store class labels here

**Exercise**: Now let's set up a simple classifier. For this exercise, we're going to use a *Logistic Regression* classifier, as you might have encountered during the earlier lecture.

For details, you can take a look at the scikit-learn documentation for [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). For now, use the default parameters, then train the algorithm on the training data and explore how well it does using the test data (take a look at the `score` method of the Logistic Regression classifier). Do you think it produces good results? 

**Hint**: Basically all algorithms implemented in scikit-learn have the same interface. Basically all of them have a `fit` method that will fit your data, a `predict` method that will predict the classes/values of new samples, and a `score` method that tells you something about how good your algorithm is at making predictions. Some algorithms also have a `transform` and a `fit_transform` method, which allows you to transform your features (e.g. dimensionality reduction algorithms like Principal Component Analysis). Many machine learning libraries outside of scikit-learn have adopted the same structure, which is super helpful when using these libraries for algorithms that are not implemented in scikit-learn. Learning how to do a workflow in scikit-learn is well worth the investment.

So the accuracy on the training is very low, but it seems like the model does a *perfect* job on the test data set and manages to classify *every single* training example correctly (at least in my version). Pretty cool, right? 

Well, no. In general, a model *cannot* do better on examples that it has never seen before than on samples it *has* seen before. A test (or validation) score that is perfect (or at least significantly better than the training score) should make us suspicious. That the optimizer complained at us doesn't help, either.

**Exercise**: Take five minutes and thing about why this might happen. Take a look at the training and the test data. Are there any notable differences? 

**Hint**: You might want to look at the class probabilities that the Logistic Regression classifier returns. You can also plot distributions of samples and labels for the two different datasets.

So it looks like the model only returns probabilities of 0.5 for all of our examples! This is obviously not great; it suggests that the model knows *nothing* about what the samples in our test data are. Whether you get a prediction for a STAR or a GALAXY depends *entirely* on what the programmers of scikit-learn decided what the algorithm should return in case that the probabilities are 0.5. We might just as well have gotten back a score of 0 if they'd made a different choice. 

Obviously, this is bad: we *thought* that the algorithm had learned something useful, but it hadn't. But why? 

Well, when I generated the data, I did a silly thing: 
* I downloaded 50,000 stars and 50,000 galaxies from SDSS
* I concatenated both together and stored them in a file
* Then we took the first 51,000 examples from that file and called it training data, and called the rest test data.

What does that do? Display the number of samples labelled "STAR" and those labelled "GALAXY" in both the training and test data set to find out. :) 
Hint: There are many ways you could do this, but casting the training and test labels into a `pandas.Series` object and using the `value_counts()` function gives you a pretty straightforward way of doing this.

You can do this by hand, of course, but the scikit-learn function `train_test_split` is very useful here. Let's use that instead:

**Exercise**: Let's train our logistic regression classifier again and see how it goes this time:

That looks a lot better, but still not great. This, in part, is because we left out a bunch of crucial steps. Remember how I said **look at the data first**? Yeah, let's do a better job of this and start from the beginning:



# Machine Learning: From Start To Finish

I set a really bad example above by (1) asking you to run the algorithm on the data before we'd even looked at it, and (2) using the test data set before we were ready. The idea was to give you an idea to explore some of the things that can go wrong with your machine learning if you're not careful. We're now going to do better, and run through a typical machine learning workflow from start to finish.


## Figuring out your goal: Asking why!

The first step in *any* research project, but certainly in any machine learning project, is to define your goal. What are you going to do with your results? Are you trying to learn something about physics with your data set? Are you just trying to separate out good signals from bad ones? 

Unfortunately, many physics questions don't necessarily easily translate to a machine learning problem (this is often true for statistics as well). Going from "I want to know what dark matter is" to "run a random forest on SDSS photometric measurements" is really hard, and requires multiple iterations of reducing your *physics* question down to one that ML or statistics (or a combination of the two) can answer. It's well worth spending significant time at this stage, because this process will give you crucial insights about every step of your analysis procedure, from which columns in your data might be useful, to which dimensionality reduction algorithms might work, to the type of appropriate ML algorithm to use. 

One important question I've already implicitly answered for you above: Do you have ground-truth labels in your training data? That is, do you have data for which you are reasonably sure that the labels assigned to each sample are correct? Another question I've already implicitly answered: Do you have a **classification** or a **regression** problem. In a classification problem, you have categorical labels (e.g. "star", "galaxy", ...) that you try to assign to new samples. In a regression context, the variable you try to assign is continuous (e.g. redshift). 

Here are some additional questions you might want to think about at the very start, but also keep in mind throughout your analysis:
* Do you only care about the *predictions*, or do I also care about the *parameters*? That is, are you trying to learn something about the structure of the problem itself (e.g. physics), or do you just care that the predictions are right?
* How well does your training data match the samples for which you don't know the labels? Are they from the same survey/instrument? Are there significant differences between the data you can train on, and the data you want to classify? 
* What biases do you already know if in your training data? Is your survey flux-limited? Did the team making the catalogue only care about a particular funny type of Cataclysmic Variable and leave out another you might be interested in? Is there a part of feature space that's just not covered? 
* What physical knowledge do you have about your measurements? How can that physics knowledge guide you in selecting or constructing good features? 



## Feature Selection and Engineering

One of the crucial parts of machine learning, and the part that you will likely spend most of the time on, is selecting and engineering features for training on. Features are, essentially, meaningful summaries of your data that are ideally designed such that they make classification and regression easy. In terms of the problems you're considering today, notice that the magnitudes we've extracted from the SDSS catalogue are not actual data. They're measurements derived from the *images* that the telescope took. 

There are machine learning algorithms that can take raw data, e.g. the pixel values in an image. The most popular type used especially in image recognition in recent years is the Convolutional Neural Network (CNN), which takes raw data and essentially internally learns what good representations of the data are.

CNNs have been enormously successful for a whole number of tasks, but it's worth pointing out that they're not always the ideal solution. They're big and very, very expensive to train (some of them can take weeks even on supercomputers!). If you have knowledge about the structure of your data and your problem, then it may be more efficient and reliable to use that knowledge to extract meaningful features. For example, we know that photons hit a CCD following a point spread function (PSF). We might not know that PSF very well, but if we do, there's no point making a neural network *learn* that a PSF exists; we can just extract magnitudes and work with them directly. So for any ML problem, it's worth thinking about what you know, and what you don't know. In cases where you *don't* know your PSF very well (or any number of things that may affect your measurements), it might be worth having your model learn that structure, but just take note that that's not universally true.

Let's now turn to the data you extracted. Earlier, you've taken a look at columns and might already have some ideas about which columns are useful for the star/galaxy classification project.

In any data set, you might have columns that are more useful than others! Let's find the ones we want to use for classification!

**Exercise**: Print the columns in your features table. What columns are there in your table? Are all of them useful for solving your problem? Discuss with your team which ones might be useful, and which ones might not be! Reminder: you can look up what some of those columns mean [here](http://skyserver.sdss.org/dr14/en/help/browser/browser.aspx#&&history=description+SpecPhoto+V). 

**Exercise**: drop all columns that you don't think are useful.


**Note**: For this exercise, I've made you columns that contain the colours as potentially useful classification features, but these might not exist in data sets that you encounter in research contexts. 

**Hint**: Feature engineering might include *combining* measurements into new features based on your domain knowledge of the problem you're trying to solve.

**Note**: for now, let's leave the `class` column in the data, which will help us with visualization later:


### Visualizing Feature Spaces

One of the most useful things you can do to star with is visualize your data! There are, of course, many different ways to visualize high-dimensional data. For example, you can make a histogram of the distributions for each feature (for classification problems, colour-coded by class works well), or you can make scatter plots or heatmaps of one feature against one another. You can also do both at the same time in what's called a *corner plot* or a *pair plot*. In python, the package [`seaborn`](https://seaborn.pydata.org/index.html) has a lot of nice visualizations pre-built to cut down the time you need to deal with nitty-gritty details of making complicated plots work.

**Exercise**: Discuss and try out ways to visualize your features. Take a look at the [`pairplot`](https://seaborn.pydata.org/examples/scatterplot_matrix.html) function in seaborn. In particular, it can be useful to set `hue="class"` to automatically plot separate distributions for `STAR` and `GALAXY` samples. For the classification problems, look for features that separate stars and galaxies well. For the regression, try to gauge by looking at the features how complex a model you'll likely need.

**Note**: a pairplot for *all* of the features will be pretty big: pick a subset you think might be informative with respect to classification, and then do a pairplot of those. It's also helpful to start by just plotting the first 1000 samples in your DataFrame if `pairplot` runs very slowly, and to set `diag_kind = "kde"`

**Hint**: This is a good time to scramble the order of the data, so let's do that first:

In [103]:
data = data.sample(frac=1, replace=False).reset_index(drop=True)

You should see that there are values around -10000 for the colour and some of the magnitudes. This is because SDSS uses `-10000` as their placeholder for bad measurements, equivalent to the `NaN` ("not a number") you might see in other datasets. 

Let's remove all samples that have any -10000 value (there are ways to deal with missing data in machine learning, but we'll not learn about those today):

### Looking for Weird Things

One of the key tasks during this stage of your analysis is to *sanity-check* your data. Are there weird things? Instrumental artifacts? Things that don't look right? This is where you explore your data and try to find (and explain) these things, and potentially remove them before training.

Our star/galaxy data looks reasonably ok at this point, there are no longer any magnitudes that are wildly different from what we'd expect from either stars or galaxies in an optical survey like SDSS.



### Separate Out the Classes Again + Setting up training/testing splits

At this point, we should remove the 'class' column from the data again and set up our train/test splits. 

From now on, we'll *only* explore things on the training data, and ignore the test data until the very end.

Another important thing to think about is **stratification**: if your data set is very imbalanced, randomized splitting into training and test sets can lead to all examples of the smaller class ending up in one of the two sets, but not in both. Stratified splitting uses the class labels to ensure that examples of all classes end up in the training and test sets.



In [117]:
classes = data_clean["class"]

In [118]:
X_train, X_test, l_train, l_test = train_test_split(data_clean, classes, train_size=0.75, shuffle=True, stratify=None)
X_train = X_train.drop(["class"], axis=1)
X_test = X_test.drop(["class"], axis=1)


### Dimensionality Reduction

Another way to try and make sense of high-dimensional feature spaces is to perform *dimensionality reduction*. There are a lot of different ways to reduce the dimensionality of your features, and some are more useful than others, depending on the structure of your data and your problem. One idea with dimensionality reduction is to find the combination of features that gives you the most useful information in terms of your regression or classification problem. Some other times, all you want is a handy visualization in 2D, since humans in general aren't very good at thinking in higher-dimensional spaces.

[Principal Component Analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis) is one of the most straightforward ways of reducing dimensionality (see also [here](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition) for alternative methods implemented in scikit-learn). One useful thing to try might be to project your features into a smaller space and see whether you still capture the information relevant in order to perform a good classification or regression.

**Exercise**: Use PCA to project your features into 2 dimensions, and compare with the full data space. Do you think most of the information in your features is captured by the 2D representation? 

**Hint**: The `fit_transform` method will come in handy here.

Another way to visualize high-dimensional data is called *t-distributed stochastic neighbour embedding*, or, for short, t-SNE. [The paper](https://lvdmaaten.github.io/publications/papers/JMLR_2008.pdf) is surprisingly readable for a computer science paper, and the method is generally pretty good at visualizing complex feature spaces (and you can spend some entertaining minutes letting your brain find fun things in the patterns it produces). 

One thing you have to be aware of, however, is that t-SNE *does not generalize to new examples*. Wheras methods like PCA can be trained on some data, and then the trained model applied to new samples, this is *not true* for t-SNE. So this is a method that's useful for visualization, but it doesn't necessarily produce features you want to use in your classification. Note that t-SNE is generally quite a slow algorithm, so running this on very large data sets might leave you waiting for a while!

**Exercise**: Let's try it out! Visualize your feature space in 2 dimensions using t-SNE (hint: this is also implemented in scikit-learn).

### Scaling Features

Sometimes, your features vary wildly in order of magnitude. You may have photometric magnitudes that all lie between 13 and 20, but maybe you have a feature that's between 1 and 1,000,000, so that your different dimensions have vastly different scales. Some algorithms (e.g. random forests) can deal with that pretty well, but others can't. It's worth knowing enough about the algorithms you use whether they will deal with this kind of issue or not. If the method can't, one common solution is to re-scale the features such that they all have a mean of zero and a variance of one. 

Scikit-learn has a few ways of scaling features and other potentially useful steps for pre-processing data. Take a look at [this tutorial](http://scikit-learn.org/stable/modules/preprocessing.html). 

**Exercise**: Do you think scaling will help in your problem? Try scaling your features and re-running the PCA and t-SNE algorithms. Did your results change?

## Picking a Machine Learning Algorithm to Start With

Which machine learning algorithm you choose for your problem depends, as we've discussed above, strongly on the type of problem you're trying to solve and the structure of the data. But even once you've decided whether you have a regression or classification problem, and whether it's an unsupervised or supervised one, there remain many algorithms to choose from!

For classification, bs you learned earlier this week, there are two different types of algorithms to keep in mind: *generative* algorithms and *discriminative* algorithms. Generative algorithms are named this way because they can *generate* data. A popular example is Gaussian Mixture Models (GMMs). Discriminative models draw more or less complex functions in a multi-dimensional space (the number of dimensions corresponds to your number of features). Broadly, in the regression case, these methods try to find the best function to draw *through* the data points in order to model them. In the classification case, these methods try to find a surface that *separates* the different classes from another (this is also called a decision boundary). 

One big question is how *interpretable* the model ought to be. Simpler algorithms are often easier to understand and the results more straightforward to interpret than from, say, a random forest. A general good suggestion is to start with the simplest model you think you can get away with, and only move to more complex models if your problem demands them.

Neural networks have been hugely successful in solving complex machine learning problems, in part because they can *learn* features rather than require the user to hand-craft them. While they do very well in many circumstances, be aware that this isn't *always* the case! These networks work really well (especially for image recognition) when you have no idea what your features ought to be, or there are things in your data that you can't model very easily (or that might be very expensive to model). However, if you understand the underlying structures in your data really well, then extracting physically meaningful features related to your problem can make your algorithm outperform those that need to learn these structures from scratch, or be faster to run.

There is another side to this, though: if you *don't* know structures in your data, then neural networks can be very powerful *emulators* of the process you're trying to model. This has been used, for example, for modelling detectors, where running physical simulations of these detectors would be hugely expensive.

**Exercise**: You've earlier tried out logistic regression as an algorithm. It's time to revisit this choice. Do you think it was a reasonable one? Given your data explorations earlier, do you think you could get away with fitting a linear model (i.e. a method linear in the parameters)? Or is your function complex enough that you'll need to fit a more complicated function? Note that for many problems, there isn't a single right answer. [This slightly tongue-in-cheek flow chart](http://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) might give you a starting point which methods to explore (Note that in my science projects, I very often end up in the field labelled "tough luck"). 

Some fun methods to try:
* Logistic Regression (linear)
* Support Vector Machines (linear)
* Decision Trees + Random Forests


### Hyperparameters

Before we go into actually applying a machine learning algorithm to our data, there is one more thing we need to think about: the parameters! Here's where it gets a little confusing, though, so bear with me. All maachine learning algorithms have parameters. For example, a neural network has *weights* for each of the connections between network nodes. However, basically all algorithms also have *hyperparameters*. For the K-Nearest Neighbour algorithm, one hyperparameter is the number of neighbours to use in order to determine the model value. For a random forest, one hyperparameter is the number of trees. 

**Exercise**: Take a look at the hyperparameters for the algorithm you picked above. Which do you think are particularly useful? Are there any that don't make sense to you at all? 

### K-Fold Cross Validation

Okay, so we've established that basically all machine learning methods have *hyperparameters* we somehow need to deal with.

What to do with all those hyperparameters? One way to deal with them is *model selection*. Each different set of hyperparameters defines a different models, which you can compare. To compare, you'll to *score* your model in some way, i.e. determine how good it is. What "good" means depends on your problem at hand. Above, you used the `score` method for the K-Nearest Neighbour method, which by default uses *accuracy*, i.e. the fraction of correctly identified samples in your data set. We've also seen above that accuracy isn't always the best option, depending on what you're trying to find out.

Another key piece of information is that there's a reasonably high *variance* in whatever score you compute. That is, a random slice into training and test data will produce different scores than a second random slice. One way to deal with this is called cross-validation. There are several kinds, but the most common one we'll discuss here is called **K-Fold Cross Validation**. Under this scheme, you split your training data set (*after* you've already set aside a test set!) into $K$ different slices. In the first instance, you train on nine of these slices, and test on the tenth. In the second instance, you train on slices 1-8 and 10 and test on 9. You continue this process until each of your slices has been used to test the performance. To formally distinguish these slices from your test set, these are usually called **validation** sets, because you use them to *validate* your model, and choose hyperparameters. 

**Note**: For problems with imbalanced class distributions, there's also a useful scikit-learn class called `StratifiedKFold`. Using this to set up your k-fold cross validation will ensure that all the subsets of data created by the k-fold cross validation always contain a proportional fraction of all classes. Otherwise, especially with classes that are rare in your data set, you might end up with training or validation sets that contain *no* examples of a class you're interested in.

**Exercise**: Check the hyperparameters for the algorithm you intend to use. Normally, you should try to learn enough about your algorithm in order to make some educated guesses for what those hyperparameters should be based on your problem, your data and the algorithm itself. In this case, however, we are going to use cross-validation in order to figure out which ones you should use. So pick a parameter you think might be important to your model's performance, and run cross-validation for multiple different values of this. Print or plot the resulting scores.

**Hint**: In `scikit-learn`, you can use e.g. [`cross_val_score`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score) in order to evaluate models using cross-validation. You can learn more about evaluating the estimator performance with cross validation [here](https://scikit-learn.org/stable/modules/cross_validation.html).

So if you run a 5-fold cross-validation, this function returns 4 accuracy scores for the subset of data it trained on. This means that the computer split the data set inot 5 subsets. It then trained on the first four, and calculated a validation score on the last. Then it did the same again, but left out the second-to-last subset to use as a validation set, and so on, until every subset had been used once as a validation set. You can use more or fewer folds within the cross validation, but it's useful to remember: fewer folds of cross-validation means that you will be more uncertain about your scores (it's harder to see what the variance in scores is), but with more folds, there are fewer training examples per fold, which might degrade the performance.


**Exercise**: Write a loop that calculates cross-validated scores for a list of different values of `C`:

**Hint**: You'll want explore values of `C` over multiple orders of magnitude. That's usually better done on a logarithmic scale.

## Exploring the Results

Now is the time to explore the results: where did the algorithm do well? Are there parts of parameter space that the algorithm systematically gets wrong?

Here, you'll want to split your training data into a single training and single validation set using `test_train_split`, instead of doing cross-validation, so that you can explore the performance on a single validation set in more detail. 

Things to explore:
* plots of different features coloured by whether the algorithm successfully classified samples or not --> are there parts of parameter space where it does badly?
* make a [confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) (this is especially useful on problems with more than two classes :) )
* We've not talked a lot about scoring functions in this tutorial, but in reality, you might find that accuracy, AKA "how many samples were classified correctly" can be a bit of a limiting performance estimator in real circumstances. Take a look at [this tutorial](https://scikit-learn.org/stable/modules/model_evaluation.html) on scoring functions for more information.

Based on what you learn during this exploration, you might want to 
1. do some more hyper-parameter optimization
2. explore a different set of features (maybe a subset, or some scientifically meaningful combination of features)
3. try a different algorithm
4. Celebrate your success. :)

### Share Your Results!

You can play around with algorithms, hyperparameters, subsets or combinations of features. When you're done, add your results to the [SOSTAT 2021 ML Exercise Leaderboard](https://board.net/p/sostat2021-ml-leaderboard). Who will get the highest accuracy?